In [284]:
import pandas as pd
import os
import matplotlib.pyplot as plt
%matplotlib inline
import time
import math
import numpy as np
import datetime as dt
from datetime import date, datetime, timedelta

In [105]:
failures_df = pd.read_csv('PdM_failures2.csv',index_col=0)
telemetry_df_days = pd.read_csv('processed_telemetry.csv',index_col=0)

In [106]:
telemetry_df_days_scale = telemetry_df_days.copy()
for i in range(len(telemetry_df_days)):
    telemetry_df_days_scale['volt'][i] = telemetry_df_days['volt'][i]/telemetry_df_days['volt'].max()
    telemetry_df_days_scale['rotate'][i] = telemetry_df_days['rotate'][i]/telemetry_df_days['rotate'].max()
    telemetry_df_days_scale['pressure'][i] = telemetry_df_days['pressure'][i]/telemetry_df_days['pressure'].max()
    telemetry_df_days_scale['vibration'][i] = telemetry_df_days['vibration'][i]/telemetry_df_days['vibration'].max()
#telemetry_df_days_scale.to_csv('processed_telemetry_scaled.csv')

c:\users\olfoa\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
c:\users\olfoa\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
c:\users\olfoa\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentatio

In [109]:
telemetry_df_days_scale

,day_num,date,machineID,volt,rotate,pressure,vibration
0,1,2015-01-01,1.0,0.778868,0.907848,0.635519,0.653450
1,2,2015-01-02,1.0,0.773930,0.924974,0.656522,0.648729
2,3,2015-01-03,1.0,0.785385,0.916093,0.658057,0.857631
3,4,2015-01-04,1.0,0.784867,0.901917,0.647500,0.851501
4,5,2015-01-05,1.0,0.783207,0.947316,0.680531,0.647902
...,...,...,...,...,...,...,...
36495,361,2015-12-27,100.0,0.788497,0.900757,0.645044,0.653438
36496,362,2015-12-28,100.0,0.776419,0.946923,0.655262,0.649552
36497,363,2015-12-29,100.0,0.759835,0.920683,0.621358,0.672482
36498,364,2015-12-30,100.0,0.761920,0.936885,0.666915,0.647039


In [113]:
def stddev_series(column,period):
    sdev_series=[0.0]*(period)
    for i in range(period,len(column)):
        sdev = column[i-period:i].std()
        sdev_series.append(sdev)
    return(sdev_series)



In [ ]:
measurements = ['volt', 'rotate', 'pressure', 'vibration']
n_per = 5
IDs = list(telemetry_df_days_scale['machineID'].unique())

telemetry_df_dict = {}
for id in IDs:
    telemetry_df_dict[id] = telemetry_df_days_scale[telemetry_df_days_scale['machineID']==id].reset_index(drop=True)    

for id in IDs:
    for measurement in measurements:
        telemetry_df_dict[id][measurement + ' sdev '+str(n_per)] = stddev_series(telemetry_df_dict[id][measurement],n_per)

In [125]:
failures_df_dict = {}
for id in IDs:
    failures_df_dict[id] = failures_df[failures_df['machineID']==id].reset_index(drop=True)



In [ ]:
telemetry_df_dict1=telemetry_df_dict.copy()

In [201]:
telemetry_df_dict[1].head()

,day_num,date,machineID,volt,rotate,pressure,vibration,volt sdev 5,rotate sdev 5,pressure sdev 5,vibration sdev 5
0,1,2015-01-01,1.0,0.778868,0.907848,0.635519,0.653450,0.0,0.0,0.0,0.0
1,2,2015-01-02,1.0,0.773930,0.924974,0.656522,0.648729,0.0,0.0,0.0,0.0
2,3,2015-01-03,1.0,0.785385,0.916093,0.658057,0.857631,0.0,0.0,0.0,0.0
3,4,2015-01-04,1.0,0.784867,0.901917,0.647500,0.851501,0.0,0.0,0.0,0.0
4,5,2015-01-05,1.0,0.783207,0.947316,0.680531,0.647902,0.0,0.0,0.0,0.0


In [200]:
failures_df_dict[1].head()

,datetime,machineID,failure,date,days_last_maint,age,model
0,05/01/2015 06:00,1,comp4,2015-01-05,173,18,model3
1,06/03/2015 06:00,1,comp1,2015-03-06,45,18,model3
2,20/04/2015 06:00,1,comp2,2015-04-20,323,18,model3
3,19/06/2015 06:00,1,comp4,2015-06-19,135,18,model3
4,02/09/2015 06:00,1,comp4,2015-09-02,75,18,model3


In [209]:
for id in IDs:
    telemetry_df_dict[id]['fail'] = [0]*len(telemetry_df_dict[id])
    for date1 in telemetry_df_dict[id]['date']:
        for date2 in failures_df_dict[id]['date']:
            if date1==date2:
                index = telemetry_df_dict[id][telemetry_df_dict[id]['date']==date1].index
                telemetry_df_dict[id]['fail'][index]=1

c:\users\olfoa\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [213]:
telemetry_df_dict[3][telemetry_df_dict[3]['fail']==1]

,day_num,date,machineID,volt,rotate,pressure,vibration,volt sdev 5,rotate sdev 5,pressure sdev 5,vibration sdev 5,fail
6,7,2015-01-07,3.0,0.777318,0.924442,0.662238,0.645237,0.007303,0.087183,0.016722,0.008692,1
36,37,2015-02-06,3.0,0.761128,0.879664,0.688751,0.648961,0.055790,0.023503,0.008630,0.019645,1
201,202,2015-07-21,3.0,0.770979,0.930616,0.682806,0.684063,0.013548,0.084143,0.013217,0.016273,1
276,277,2015-10-04,3.0,0.782193,0.903026,0.646378,0.644773,0.011793,0.123323,0.009009,0.014925,1
336,337,2015-12-03,3.0,0.766495,0.897005,0.640236,0.676331,0.011266,0.088511,0.015805,0.023038,1


In [188]:
machine1_telemetry[machine1_telemetry['date']=='2015-01-05']['fail'].index

Int64Index([4], dtype='int64')

In [180]:
machine1_telemetry[machine1_telemetry['date']=='2015-01-05']['fail']

4    0
Name: fail, dtype: int64

In [295]:
sum_sdev_all = []
sum_sdev_fail = []
measurements = ['volt', 'rotate', 'pressure', 'vibration']

for measurement in measurements:
    for id in IDs:
        sum_sdev_all.append(telemetry_df_dict[id][5:][telemetry_df_dict[id][5:]['fail']==0][measurement +' sdev 5'].mean())
        sum_sdev_fail.append(telemetry_df_dict[id][5:][telemetry_df_dict[id][5:]['fail']==1][measurement + ' sdev 5'].mean())
        sum_sdev_fail = [0 if math.isnan(x) else x for x in sum_sdev_fail]

    print(measurement + ' sum_sdev_nofail: ' +str(np.mean(sum_sdev_all)))
    print(measurement + ' sum_sdev_fail: ' +str(np.mean(sum_sdev_fail)))
    print('\n')

volt sum_sdev_nofail: 0.016464265057376384
volt sum_sdev_fail: 0.026434945964707676


rotate sum_sdev_nofail: 0.021382212235426818
rotate sum_sdev_fail: 0.03823562405263391


pressure sum_sdev_nofail: 0.020544763935115964
pressure sum_sdev_fail: 0.034834149077145646


vibration sum_sdev_nofail: 0.021001386913125645
vibration sum_sdev_fail: 0.034927683757892




#on average the standard deviation of each measurement is greater in the 5 days before failure, than in other 5 day periods without failure